# Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [6]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./tools/input/fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./tools/input/fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:03<00:00,  3.77s/it]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [32]:
# from IPython.display import Markdown, display
# display(Markdown("./tools/input/fake_resume.md"))

## Creating Agents

In [33]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [34]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [35]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [36]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [37]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [41]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [42]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="./output/tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [43]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="./output/interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [45]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

2025-04-20 18:44:37,669 - 4476802560 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [47]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [48]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

- Dislplay the generated `tailored_resume.md` file.

In [51]:
from IPython.display import Markdown, display
display(Markdown("./output/tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a visionary Software Engineering Leader with 18 years of experience, specializing in artificial intelligence, data science, and strategic team leadership. With a proven track record in leading both remote and in-office teams, Noah is adept in driving technological innovation and fostering high-performing engineering teams. His deep expertise in programming languages including Python, Ruby, and Elixir, combined with his MBA and strong leadership in deploying scalable AI solutions, positions him as a top candidate for executive roles in leading tech organizations.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division into a central revenue generator, integrating AI and machine learning to enhance product capabilities and market competitiveness.
- Led strategic AI initiatives that significantly influenced the company's long-term technology roadmap and industry standing.
- Oversaw a major expansion of the engineering team, enhancing skill sets aligned with cutting-edge technology trends.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Orchestrated engineering strategies and operations, aligning with C-level visions to propel the company's technological advancements.
- Championed a culture of excellence and innovation across global teams, significantly boosting performance and project delivery.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Launched a tech startup focused on IoT solutions for pet care, introducing a market-leading GPS tracking collar.
- Directed all phases of product development, establishing a strong foundation for successful market entry and operational scalability.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Developed strategic plans improving operational collaboration, leading to enhanced project outcomes and innovation.
- Implemented advanced software development practices, significantly contributing to the company's product success.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led a high-performance team in leveraging data analytics for strategic business decisions, increasing revenue by 500%.
- Pioneered the development of predictive modeling tools that transformed customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Enhanced a central API, boosting system performance and user experience for a large-scale engineering operation.

### BetCraft: CTO — 2013 - 2015
- Guided technological strategy and execution post-Series A, enhancing platform capabilities and market expansion.

## Skills
- Expert in Python, Ruby, Elixir
- Advanced knowledge in AI, Machine Learning, and Data Science
- Strategic Leadership and Team Management
- Agile and Scrum Methodologies
- IoT and Cloud Technologies

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is uniquely qualified for senior executive roles within technology-driven enterprises, bringing a blend of technical mastery and strategic acumen to foster innovation and growth.

- Dislplay the generated `interview_materials.md` file.

In [52]:
display(Markdown("./output/interview_materials.md"))

### Interview Questions and Talking Points for Noah Williams

#### Leadership and Strategic Direction
1. **Question:** Can you describe a specific instance at DataKernel where your strategic initiatives significantly influenced the company's technology roadmap?
   - **Talking Point:** Highlight the integration of AI technologies and the focus on scalable vector databases, discussing how these decisions were aligned with long-term goals.

2. **Question:** As a founder and CEO of InnovPet, what were the key challenges you faced while leading the company from concept to market entry, and how did you overcome them?
   - **Talking Point:** Discuss the strategic decision-making and leadership skills utilized in setting up advisory boards, establishing production facilities, and navigating initial funding phases.

#### Technical Proficiency and Project Management
3. **Question:** How have your deep expertise in Python, Ruby, and Elixir contributed to the success of major projects like crewAI and active_model_serializers?
   - **Talking Point:** Explain the technical challenges and the innovative solutions implemented, emphasizing your hands-on approach and problem-solving skills.

4. **Question:** You've managed remote teams across DataKernel and InnovPet. What strategies have you implemented to ensure high performance and morale among distributed teams?
   - **Talking Point:** Share insights on fostering a culture of transparency, continuous improvement, and how you align team goals with organizational objectives.

#### Innovation and Growth
5. **Question:** In your role at EliteDevs, you were involved in setting strategic plans that improved operational collaboration. Can you discuss how these strategies impacted innovation and project outcomes?
   - **Talking Point:** Provide examples of how inter-departmental coordination was enhanced and the positive effects it had on innovation and project delivery.

6. **Question:** What role did data science play in your leadership at PrintPack, and how did it contribute to business growth?
   - **Talking Point:** Elaborate on the development of predictive modeling tools and their impact on customer behavior analysis and revenue growth.

#### Community Involvement and Mentorship
7. **Question:** With your involvement in various tech communities and open-source projects, how do you balance community engagement with your professional responsibilities?
   - **Talking Point:** Discuss the importance of community involvement in your career, the benefits it brings, and how it influences your leadership style.

#### Adaptability and Learning
8. **Question:** How has your MBA and continuous learning in AI and data science helped you stay ahead in the rapidly evolving tech industry?
   - **Talking Point:** Reflect on the impact of your advanced education on your leadership capabilities and decision-making processes in tech environments.

These questions and talking points are designed to help Noah Williams effectively showcase his leadership qualities, technical expertise, and strategic vision during his interview for a senior executive role in technology-driven enterprises.